In [1]:
# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory
import seaborn as sns
import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(1991)
torch.manual_seed(1991)
torch.cuda.manual_seed(1991)

import random
random.seed(1991)

import numpy as np
np.random.seed(1991)

Running on device: CUDA


In [2]:
# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:03<00:00, 43298769.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Unlearning algorithm
def unlearning(net, retain, forget, validation, epochs):
    """Unlearning by fine-tuning.

    Fine-tuning is a very simple algorithm that trains using only
    the retain set.

    Args:
      net : nn.Module.
        pre-trained model to use as base of unlearning.
      retain : torch.utils.data.DataLoader.
        Dataset loader for access to the retain set. This is the subset
        of the training set that we don't want to forget.
      forget : torch.utils.data.DataLoader.
        Dataset loader for access to the forget set. This is the subset
        of the training set that we want to forget. This method doesn't
        make use of the forget set.
      validation : torch.utils.data.DataLoader.
        Dataset loader for access to the validation set. This method doesn't
        make use of the validation set.
    Returns:
      net : updated model
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    net.train()

    for _ in range(epochs):
        for inputs, targets in retain:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        scheduler.step()

    net.eval()
    return net


In [4]:
from sklearn.linear_model import LogisticRegression

def train_mia_on_class(model, class_index, train_set, test_set, no_class=True, splits_num=5):
  model = model.to("cpu")

  if not no_class:
    # Select indencies where class is class_index
    train_class_set = np.where(np.array(train_set.targets) == class_index)[0]
    test_class_set = np.where(np.array(test_set.targets) == class_index)[0]
  else:
    train_class_set = np.arange(len(train_set))
    test_class_set = np.arange(len(test_set))

  max_len = min([len(train_class_set), len(test_class_set)])
  # Make equal sizes
  train_class_set = train_class_set[:max_len]
  test_class_set = test_class_set[:max_len]

  # Obtain subsets
  train_class_set = torch.utils.data.Subset(train_set, train_class_set)
  test_class_set = torch.utils.data.Subset(test_set, test_class_set)


  # Make them
  class_test_loader = torch.utils.data.DataLoader(
    test_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  class_train_loader = torch.utils.data.DataLoader(
    train_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  criterion = nn.CrossEntropyLoss(reduction="none")
  # Obtain train and test logits
  logits_train = []
  for i, (images, labels) in enumerate(class_train_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_train.append(np.concatenate((logits.detach().numpy()[0], losses)))

  logits_test = []
  for i, (images, labels) in enumerate(class_test_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_test.append((np.concatenate((logits.detach().numpy()[0], losses))))

  logits_train, logits_test = np.array(logits_train), np.array(logits_test)
  # Create dataset
  ys = [1] * max_len + [0] * max_len
  ys = np.array(ys)
  p = np.random.permutation(len(ys))
  logits = np.concatenate((logits_train, logits_test))

  logits = logits[p]
  ys = ys[p]

  # Fit logitstic regression
  clf = LogisticRegression(random_state=0, max_iter=1000)
  cv = model_selection.StratifiedShuffleSplit(
        n_splits=splits_num, random_state=0
    )

  clf = model_selection.cross_val_score(
        clf, logits, ys, cv=cv, scoring="accuracy"
    )

  # Output score
  # print(f"Mean MIA attack score for class {class_index} is: " + str(clf.mean()))
  return clf.mean()

In [5]:

# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader, 0):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total

In [6]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)


### Plain network without regularization
### 5 % of dataset class; 5 epochs

In [10]:

print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Baseline"
model_params = "cifar10_resnet18_baseline.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader, 5)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Baseline
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 94.8
MIA score before on forget set: 0.88


Retain set accuracy FORGET model: 99.4%
Test set accuracy FORGET model: 76.6%
Forget set accuracy FORGET model: 85.2%

The MIA has an accuracy of 0.892 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 98.4
MIA score before on forget set: 0.94


Retain set accuracy FORGET model: 92.9%
Test set accuracy FORGET model: 75.0%
Forget set accuracy FORGET model: 84.8%

The MIA has an accuracy of 0.924 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 89.60000000000001
MIA score before on forget set: 0.87


Retain set accuracy FORGET model: 98.7%
Test set accuracy FORGET model: 76.1%
Forget set accuracy FORGET model: 67.2%

The MIA has an accuracy of 0.860 on

In [11]:
# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.88, 0.94, 0.87, 0.84, 0.92, 0.9299999999999999, 0.87, 0.9, 0.9, 0.9]
[0.892, 0.924, 0.86, 0.776, 0.8959999999999999, 0.868, 0.8800000000000001, 0.892, 0.908, 0.8880000000000001]
[94.8, 98.4, 89.60000000000001, 84.0, 94.0, 88.0, 94.39999999999999, 96.8, 96.0, 95.6]
[99.41915385388404, 92.90925535122099, 98.74989950960689, 96.2883324625216, 99.41917720119781, 99.43924106604494, 96.82061176092287, 95.25283382908594, 96.25796338350851, 95.80553099123725]
[85.2, 84.8, 67.2, 68.4, 85.6, 73.2, 81.6, 81.2, 89.60000000000001, 86.8]
[76.63, 75.02, 76.13, 75.47, 76.53, 76.82, 75.18, 75.17, 75.67, 75.16000000000001]


In [12]:

mean_mia_score_before_baseline = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_baseline = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_baseline = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_baseline = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_baseline = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_baseline = np.mean(np.array(Accuracy_test))

In [13]:

print(mean_mia_score_before_baseline)
print(mean_mia_score_after_baseline)

print(mean_accuracy_forget_before_baseline)

print(mean_accuracy_forget_after_baseline)
print(mean_accuracy_retain_baseline)
print(mean_accuracy_test_baseline)

0.8950000000000001
0.8784000000000001
93.16
80.36
97.03619994092307
75.77799999999999


### Dropout model

In [17]:
torch.manual_seed(1991)
model_train_reg = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
model_train_reg = model_train_reg.to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_train_reg.parameters(), lr=0.001, weight_decay=1e-4)

# Decay LR by a factor of 0.001 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.0005)

In [18]:
def modify_model(model_train_reg):
  from collections import OrderedDict

  layer1 = model_train_reg.layer1

  for indx in range(len(layer1)):
    modified_layers = OrderedDict()
    for name, feature in layer1[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer1[indx] = nn.Sequential(modified_layers)



  layer2 = model_train_reg.layer2

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer2[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer2[indx] = nn.Sequential(modified_layers)

  layer3 = model_train_reg.layer3

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer3[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)
    model_train_reg.layer3[indx] = nn.Sequential(modified_layers)

  layer4 = model_train_reg.layer4

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer4[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer4[indx] = nn.Sequential(modified_layers)

  return model_train_reg

In [19]:
model_train_reg = modify_model(model_train_reg)

In [20]:
model_train_reg.load_state_dict(torch.load("cifar10_resnet18_l2_dropout_regularization.pt"))

<All keys matched successfully>

In [22]:

print("----- 5 PERCENT DROPOUT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Dropout"
model_params = "cifar10_resnet18_l2_dropout_regularization.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft = model_forget_ft.to(DEVICE)
  model_forget_ft = modify_model(model_forget_ft)

  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader, 5)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT DROPOUT -------
Model name: Resnet18 Dropout
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 86.4
MIA score before on forget set: 0.92


Retain set accuracy FORGET model: 85.1%
Test set accuracy FORGET model: 78.1%
Forget set accuracy FORGET model: 86.0%

The MIA has an accuracy of 0.880 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 85.2
MIA score before on forget set: 0.9


Retain set accuracy FORGET model: 84.6%
Test set accuracy FORGET model: 77.5%
Forget set accuracy FORGET model: 82.8%

The MIA has an accuracy of 0.920 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 67.60000000000001
MIA score before on forget set: 0.86


Retain set accuracy FORGET model: 83.2%
Test set accuracy FORGET model: 76.3%
Forget set accuracy FORGET model: 72.0%

The MIA has an accuracy of 0.

In [23]:
# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.9199999999999999, 0.9, 0.86, 0.8999999999999999, 0.96, 0.8400000000000001, 0.8300000000000001, 0.84, 0.86, 0.9299999999999999]
[0.8799999999999999, 0.9199999999999999, 0.852, 0.8719999999999999, 0.916, 0.8480000000000001, 0.8640000000000001, 0.9039999999999999, 0.884, 0.876]
[86.4, 85.2, 67.60000000000001, 57.199999999999996, 74.4, 69.19999999999999, 80.80000000000001, 80.80000000000001, 88.0, 89.60000000000001]
[85.07978616503878, 84.59822325843149, 83.2448946776009, 85.67006994131361, 85.9511606873681, 84.48088749547811, 84.32180396728099, 84.98502723234922, 84.72978676367144, 84.17604501607717]
[86.0, 82.8, 72.0, 59.599999999999994, 78.4, 68.8, 84.8, 76.0, 92.4, 86.4]
[78.12, 77.53, 76.32, 78.19, 78.42, 76.98, 77.0, 77.03, 77.71000000000001, 77.03]


In [24]:

mean_mia_score_before_dropout = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_dropout = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_dropout = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_dropout = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_dropout = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_dropout = np.mean(np.array(Accuracy_test))

In [25]:

print(mean_mia_score_before_dropout)
print(mean_mia_score_after_dropout)

print(mean_accuracy_forget_before_dropout)

print(mean_accuracy_forget_after_dropout)
print(mean_accuracy_retain_dropout)
print(mean_accuracy_test_dropout)

0.884
0.8815999999999999
77.92000000000002
78.72
84.72376852046098
77.43299999999999


### Augmentation model

In [26]:
# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
         transforms.RandomRotation(30), # Randomly rotate some images by 20 degrees
         transforms.RandomHorizontalFlip(), # Randomly horizontal flip the images
         transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1,
                                                            saturation = 0.1),
         transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
        transforms.ToTensor(),
    ]
)

to_tensor_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=to_tensor_transform
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [27]:

print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Augmentation"
model_params = "cifar10_resnet18_augmentation.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader, 5)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Augmentation
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 81.2
MIA score before on forget set: 0.93


Retain set accuracy FORGET model: 75.8%
Test set accuracy FORGET model: 76.6%
Forget set accuracy FORGET model: 79.2%

The MIA has an accuracy of 0.880 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 82.39999999999999
MIA score before on forget set: 0.9


Retain set accuracy FORGET model: 75.4%
Test set accuracy FORGET model: 76.0%
Forget set accuracy FORGET model: 80.8%

The MIA has an accuracy of 0.900 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 63.2
MIA score before on forget set: 0.81


Retain set accuracy FORGET model: 74.7%
Test set accuracy FORGET model: 75.8%
Forget set accuracy FORGET model: 60.8%

The MIA has an accuracy of 0.852

In [28]:
# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.9299999999999999, 0.8999999999999999, 0.81, 0.87, 0.8999999999999999, 0.85, 0.85, 0.85, 0.92, 0.87]
[0.8800000000000001, 0.9, 0.852, 0.8640000000000001, 0.892, 0.8480000000000001, 0.9, 0.8720000000000001, 0.8480000000000001, 0.8879999999999999]
[81.2, 82.39999999999999, 63.2, 67.2, 80.80000000000001, 67.60000000000001, 81.6, 81.2, 86.0, 78.8]
[75.80697029384572, 75.37484423362946, 74.65482243704403, 77.21170465050847, 76.8677265692521, 75.41049500572784, 76.68562471110754, 75.45724966836838, 77.48703726034005, 75.93705408284262]
[78.8, 80.4, 57.599999999999994, 55.2, 75.2, 60.4, 79.60000000000001, 76.4, 84.8, 72.39999999999999]
[76.63, 76.03, 75.82, 76.91, 77.05, 76.21, 77.73, 76.16000000000001, 77.12, 76.09]


In [29]:

mean_mia_score_before_aug = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_aug = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_aug = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_aug = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_aug = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_aug = np.mean(np.array(Accuracy_test))

In [30]:

print(mean_mia_score_before_aug)
print(mean_mia_score_after_aug)

print(mean_accuracy_forget_before_aug)

print(mean_accuracy_forget_after_aug)
print(mean_accuracy_retain_aug)
print(mean_accuracy_test_aug)

0.875
0.8744
77.0
72.08
76.08935289126661
76.575
